This notebook compares analytical and numerical evaluations of alchemical derivatives of a rotated and translated CO molecule up to the third order in KSDFT.  
The results are summerized in the last section.

In [1]:
from pyscf import gto,scf
import numpy as np
import pyscf
from scipy.spatial.transform import Rotation

pyscf.__version__

'2.2.1'

In [2]:
from aqa.FcMole import FcM_like

Rotating and translating CO with the coordinate [[0, 0, 0], [0, 0, 2.1]].

In [3]:
mol_coord = np.array([[0, 0, 0], [0, 0, 2.1]])
rotvec = np.array([2 * np.pi/1.1, np.pi/2, np.pi/3])
rot = Rotation.from_rotvec(rotvec)
mol_coord = rot.apply(mol_coord)
# print(mol_coord)
# print(np.linalg.norm(mol_coord[1]))
mol_coord[0] += [13.4, 21.9, 3.8]
mol_coord[1] += [13.4, 21.9, 3.8]
print(mol_coord)
print(np.linalg.norm(mol_coord[1] - mol_coord[0]))

[[13.4        21.9         3.8       ]
 [13.26749486 22.4300496   5.82768089]]
2.1


In [4]:
# target_mol = "C 0 0 0; O 0 0 2.1"
target_mol = "C %f %f %f; O %f %f %f" % (mol_coord[0,0], mol_coord[0,1],
                                         mol_coord[0,2], mol_coord[1,0],
                                         mol_coord[1,1], mol_coord[1,2])
dft_functional = "pbe0"  # "lda,vwn"
basis_set = "def2-TZVP"
print(target_mol)

C 13.400000 21.900000 3.800000; O 13.267495 22.430050 5.827681


### Analytical evaluation

In [5]:
%load_ext autoreload
from aqa.AP_class import APDFT_perturbator as AP

In [6]:
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
mf_nn=scf.RKS(mol_NN)
mf_nn.xc = dft_functional
mf_nn.scf()
# mf_nn.conv_tol = 1e-12
# mf_nn.grids.level = 6
# mf_nn.nuc_grad_method().kernel()
ap_nn=AP(mf_nn,sites=[0,1],flag_response_property=True)

converged SCF energy = -113.230628473572


In [7]:
analytical_1st_alchemical_derivative = ap_nn.build_gradient()
analytical_2nd_alchemical_derivative = ap_nn.build_hessian()
analytical_3rd_alchemical_derivative = ap_nn.build_cubic()

Inefficient alchemical force in KS-DFT which calculates the response matrix for nuclear coordinates

In [8]:
inefficient_analytical_alchemical_force = np.zeros((2, 2, 3))
inefficient_analytical_alchemical_force[0] = ap_nn.build_inefficient_alchemical_force(0)
inefficient_analytical_alchemical_force[1] = ap_nn.build_inefficient_alchemical_force(1)

Efficient alchemical force in KS-DFT which calculates the response matrix for nuclear coordinates

In [9]:
efficient_analytical_alchemical_force = np.zeros((2, 2, 3))
efficient_analytical_alchemical_force[0] = ap_nn.build_alchemical_force(0)
efficient_analytical_alchemical_force[1] = ap_nn.build_alchemical_force(1)

Electric dipole derivatives $\partial ^2 E/ \partial Z_I \partial \mathbf{F}$

In [10]:
elec_electric_dipole_gradient_z = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='Z')
elec_electric_dipole_gradient_f = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='F')

Electric polarizability derivatives $\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F}$

In [11]:
elec_electric_pol_gradient = ap_nn.build_electric_polarizability_gradient()

### Numerical evaluation

In [12]:
%autoreload 2
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
fc_param = 0.001
fcs1 = [fc_param, 0.0]
fcs2 = [-fc_param, 0.0]
fcs3 = [0.0, fc_param]
fcs4 = [0.0, -fc_param]

##### Efficient numerical alchemical force

In [13]:
fmol1=FcM_like(mol_NN,fcs=fcs1)
fmol2=FcM_like(mol_NN,fcs=fcs2)
fmol3=FcM_like(mol_NN,fcs=fcs3)
fmol4=FcM_like(mol_NN,fcs=fcs4)
mf=scf.RKS(mol_NN)
mf1=scf.RKS(fmol1)
mf2=scf.RKS(fmol2)
mf3=scf.RKS(fmol3)
mf4=scf.RKS(fmol4)
mf.xc = dft_functional
mf1.xc = dft_functional
mf2.xc = dft_functional
mf3.xc = dft_functional
mf4.xc = dft_functional
# mf.conv_tol = 1e-12
# mf1.conv_tol = 1e-12
# mf2.conv_tol = 1e-12
# mf3.conv_tol = 1e-12
# mf4.conv_tol = 1e-12
# mf.grids.level = 6
# mf1.grids.level = 6
# mf2.grids.level = 6
# mf3.grids.level = 6
# mf4.grids.level = 6
# Without dm0=mf.init_guess_by_1e(), some SCFs do not converge.
e=mf.scf(dm0=mf.init_guess_by_1e())
e1=mf1.scf(dm0=mf1.init_guess_by_1e())
e2=mf2.scf(dm0=mf2.init_guess_by_1e())
e3=mf3.scf(dm0=mf3.init_guess_by_1e())
e4=mf4.scf(dm0=mf4.init_guess_by_1e())

converged SCF energy = -113.230628454758


/home/takafumi/program/pyscf/pyscf/gto/mole.py:1213: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -113.24532239936
converged SCF energy = -113.215937413455
converged SCF energy = -113.252886391376
converged SCF energy = -113.208374070835


In [14]:
nuc_grad = mf.nuc_grad_method().kernel()
nuc_grad1 = mf1.nuc_grad_method().kernel()
nuc_grad2 = mf2.nuc_grad_method().kernel()
nuc_grad3 = mf3.nuc_grad_method().kernel()
nuc_grad4 = mf4.nuc_grad_method().kernel()

--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0019566885     0.0078496723     0.0300268750
1 O     0.0019264416    -0.0078534442    -0.0300279371
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0019427245     0.0077938211     0.0298132164
1 O     0.0019124796    -0.0077975932    -0.0298142812
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0019706549     0.0079055333     0.0302405707
1 O     0.0019404060    -0.0079093050    -0.0302416301
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0019537319     0.0078378486     0.0299816441
1 O     0.0019234696    -0.0078416224    -0.0299827048
----------------------------------------------
--------

In [15]:
efficient_numerical_alchemical_force = np.zeros((2, 2, 3))
efficient_numerical_alchemical_force[0] = (nuc_grad1 - nuc_grad2) / (2 * fc_param)
efficient_numerical_alchemical_force[1] = (nuc_grad3 - nuc_grad4) / (2 * fc_param)

#### Numerical alchemical derivatives

In [16]:
numerical_1st_alchemical_derivative = np.zeros((2))
numerical_1st_alchemical_derivative[0] = (e1 - e2) / (2 * fc_param)
numerical_1st_alchemical_derivative[1] = (e3 - e4) / (2 * fc_param)

#### Numerical alchemical hardness

In [17]:
ap_nn=AP(mf,sites=[0,1])
ap_nn1=AP(mf1,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn2=AP(mf2,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn3=AP(mf3,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn4=AP(mf4,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
an = ap_nn.build_gradient()
an1 = ap_nn1.build_gradient()
an2 = ap_nn2.build_gradient()
an3 = ap_nn3.build_gradient()
an4 = ap_nn4.build_gradient()
ann = ap_nn.build_hessian()
ann1 = ap_nn1.build_hessian()
ann2 = ap_nn2.build_hessian()
ann3 = ap_nn3.build_hessian()
ann4 = ap_nn4.build_hessian()
dipole1 = ap_nn1.ref_elec_electric_dipole_moment * -1
dipole2 = ap_nn2.ref_elec_electric_dipole_moment * -1
dipole3 = ap_nn3.ref_elec_electric_dipole_moment * -1
dipole4 = ap_nn4.ref_elec_electric_dipole_moment * -1
pol1 = ap_nn1.ref_elec_electric_polarizability * -1
pol2 = ap_nn2.ref_elec_electric_polarizability * -1
pol3 = ap_nn3.ref_elec_electric_polarizability * -1
pol4 = ap_nn4.ref_elec_electric_polarizability * -1

In [18]:
numerical_2nd_alchemical_derivative = np.zeros((2, 2))
numerical_2nd_alchemical_derivative[0] = (an1 - an2) / (2 * fc_param)
numerical_2nd_alchemical_derivative[1] = (an3 - an4) / (2 * fc_param)

#### Numerical alchemical hardness derivatives

In [19]:
numerical_3rd_alchemical_derivative = np.zeros((2, 2, 2))
numerical_3rd_alchemical_derivative[0, :, :] = (ann1 - ann2) / (2 * fc_param)
numerical_3rd_alchemical_derivative[1, :, :] = (ann3 - ann4) / (2 * fc_param)

#### Inefficient numerical alchemical force

In [20]:
# mol_NN1=gto.M(atom="C 0 0 0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN2=gto.M(atom="C 0 0 -0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN3=gto.M(atom="C 0 0 0; O 0 0 2.101",unit="Bohr",basis=basis_set)
# mol_NN4=gto.M(atom="C 0 0 0; O 0 0 2.099",unit="Bohr",basis=basis_set)
# mf_nn1=scf.RKS(mol_NN1)
# mf_nn2=scf.RKS(mol_NN2)
# mf_nn3=scf.RKS(mol_NN3)
# mf_nn4=scf.RKS(mol_NN4)
# mf_nn1.xc = dft_functional
# mf_nn2.xc = dft_functional
# mf_nn3.xc = dft_functional
# mf_nn4.xc = dft_functional
# mf_nn1.scf()
# mf_nn2.scf()
# mf_nn3.scf()
# mf_nn4.scf()
# ap_nn1=AP(mf_nn1,sites=[0,1])
# ap_nn2=AP(mf_nn2,sites=[0,1])
# ap_nn3=AP(mf_nn3,sites=[0,1])
# ap_nn4=AP(mf_nn4,sites=[0,1])
# ad1 = ap_nn1.build_gradient()
# ad2 = ap_nn2.build_gradient()
# ad3 = ap_nn3.build_gradient()
# ad4 = ap_nn4.build_gradient()

In [21]:
# inefficient_numerical_alchemical_force = np.zeros((2, 2))  # only z axis components
# inefficient_numerical_alchemical_force[:, 0] = (ad1 - ad2) / (2 * fc_param)
# inefficient_numerical_alchemical_force[:, 1] = (ad3 - ad4) / (2 * fc_param)

#### Numerical alchemical electric dipole moment

In [22]:
numer_elec_electric_dipole_gradient = np.zeros((2, 3))
numer_elec_electric_dipole_gradient[0] = (dipole1 - dipole2) / (2 * fc_param)
numer_elec_electric_dipole_gradient[1] = (dipole3 - dipole4) / (2 * fc_param)

#### Numerical alchemical electric polarizability

In [23]:
numer_elec_electric_pol_gradient = np.zeros((2, 3, 3))
numer_elec_electric_pol_gradient[0] = (pol1 - pol2) / (2 * fc_param)
numer_elec_electric_pol_gradient[1] = (pol3 - pol4) / (2 * fc_param)

## Comparison of analytical and numerical evaluations

$ \partial E/\partial Z_i $

In [24]:
print(analytical_1st_alchemical_derivative)

[-14.69249291 -22.25616049]


$ (\partial E/\partial Z_i)_\mathrm{analytical} $ - $ (\partial E/\partial Z_i)_\mathrm{numerical} $

In [25]:
print(analytical_1st_alchemical_derivative - numerical_1st_alchemical_derivative)

[ 4.08740437e-08 -2.23616560e-07]


$\partial^2E/\partial Z_i\partial Z_j$

In [26]:
print(analytical_2nd_alchemical_derivative)

[[-2.90329897  0.45093805]
 [ 0.45093805 -3.5526951 ]]


$(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{numerical}$

In [27]:
print(analytical_2nd_alchemical_derivative - numerical_2nd_alchemical_derivative)

[[-6.61961481e-06  1.49857844e-05]
 [-2.09104709e-06  4.97622861e-06]]


$\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [28]:
print(analytical_3rd_alchemical_derivative)

[[[-0.11581658  0.12437607]
  [ 0.12437607  0.08834224]]

 [[ 0.12437607  0.08834224]
  [ 0.08834224 -0.14458269]]]


$(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{analytical}$ - $(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{numerical}$

In [29]:
print(analytical_3rd_alchemical_derivative - numerical_3rd_alchemical_derivative)

[[[-1.40357776e-06  1.13643633e-07]
  [ 1.13643633e-07  4.35724810e-07]]

 [[-4.59286070e-07  5.79394796e-08]
  [ 5.79394796e-08  8.60743591e-08]]]


$\partial^2E/\partial Z_i\partial R_j$

In [30]:
print(efficient_analytical_alchemical_force)

[[[ 0.01396536 -0.05585711 -0.21368067]
  [-0.01396341  0.05585692  0.21367793]]

 [[ 0.00296196 -0.01184494 -0.04531227]
  [-0.0029773   0.011843    0.0453136 ]]]


$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [31]:
print(efficient_analytical_alchemical_force - efficient_numerical_alchemical_force)

[[[ 1.73203769e-07 -1.03542334e-06 -3.50162363e-06]
  [-1.89011448e-07  1.03586135e-06  3.51395997e-06]]

 [[ 8.12621789e-08 -2.02807409e-07 -1.11588799e-06]
  [-8.42610416e-08  2.03135814e-07  1.11844739e-06]]]


$\partial^2E/\partial Z_i\partial R_j$

In [32]:
print(inefficient_analytical_alchemical_force)

[[[ 0.01396536 -0.05585711 -0.21368067]
  [-0.01396341  0.05585692  0.21367793]]

 [[ 0.00296196 -0.01184494 -0.04531227]
  [-0.0029773   0.011843    0.0453136 ]]]


$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [33]:
# print(inefficient_analytical_alchemical_force[:, :, 2] - inefficient_numerical_alchemical_force)

$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$

In [34]:
print(elec_electric_dipole_gradient_f)

[[ 0.0385843  -0.1543442  -0.59043667]
 [-0.07717442  0.30871591  1.180978  ]]


In [35]:
print(elec_electric_dipole_gradient_z)

[[ 0.0385843  -0.1543442  -0.59043667]
 [-0.07717442  0.30871591  1.180978  ]]


$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$ - $(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{numerical}$

In [36]:
print(elec_electric_dipole_gradient_f - numer_elec_electric_dipole_gradient)

[[-1.42218624e-06 -2.04306816e-05 -2.19877444e-05]
 [ 1.85296078e-06  4.31660565e-06 -9.60291935e-06]]


In [37]:
print(elec_electric_dipole_gradient_z - numer_elec_electric_dipole_gradient)

[[-1.42176682e-06 -2.04307325e-05 -2.19876881e-05]
 [ 1.85341838e-06  4.31625946e-06 -9.60401990e-06]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$

In [38]:
print(elec_electric_pol_gradient)

[[[ 8.02007927  0.01754119  0.06710274]
  [ 0.01754119  7.95429173 -0.26843934]
  [ 0.06710274 -0.26843934  6.9975642 ]]

 [[ 4.27758601 -0.05521717 -0.21123132]
  [-0.05521717  4.48466385  0.84497545]
  [-0.21123132  0.84497545  7.49619384]]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$ - $(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{numerical}$

In [39]:
print(elec_electric_pol_gradient - numer_elec_electric_pol_gradient)

[[[ 8.92325635e-06  7.99068398e-07  4.10527804e-06]
  [ 7.99068398e-07  7.93011127e-06 -9.09565564e-06]
  [ 4.10527804e-06 -9.09565564e-06 -4.12437299e-05]]

 [[ 3.35024935e-06  3.32762796e-07  7.87109203e-07]
  [ 3.32762796e-07  1.26815747e-06 -6.10856578e-06]
  [ 7.87109203e-07 -6.10856578e-06 -1.22919646e-05]]]
